# **Fake News Classifier Using LSTM**

Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Fake News/train.csv')

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head(10)

In [ ]:
X = df.drop('label', axis=1)

In [ ]:
Y = df['label']

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
messages = X.copy()

In [ ]:
messages['title'][0]

In [ ]:
messages

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.stem.porter import PorterStemmer 

ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    title = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    title = title.lower()
    title = title.split()  
    title = [ps.stem(word) for word in title if not word in stopwords.words('english')]
    title = ' '.join(title)
    corpus.append(title)

In [ ]:
corpus

In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

voc_size = 5000
onehot_repr = [one_hot(words,voc_size) for words in corpus] 
onehot_repr

In [ ]:
corpus[0]

In [ ]:
onehot_repr[0]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[0]

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
embedding_vector_features = 40 
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
len(embedded_docs), len(Y)

In [ ]:
import numpy as np

X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [ ]:
X_final.shape, Y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=64)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
Y_pred

In [ ]:
Y_pred = np.where(Y_pred > 0.6, 1,0) 

In [ ]:
Y_pred

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, Y_pred)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_test, Y_pred)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))